In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_mldata
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [2]:
yeast = fetch_mldata("yeast")
X = yeast.data
Y = yeast.target.toarray().astype(np.int).T

train_test_split = np.random.rand(len(X)) < 0.70
train_x = X[train_test_split]
train_y = Y[train_test_split]
test_x = X[~train_test_split]
test_y = Y[~train_test_split]

In [3]:
training_epochs = 15
n_dimensions = train_x.shape[1]
nhu_lone = 100
nhu_ltwo = 50
n_classes = 14
learning_rate = 0.001
batch_size = 10
total_batches = train_x.shape[0] // batch_size

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 1/np.sqrt(n_dimensions))
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def fully_connected_sigmoid(x,weight,bias,return_logits = False):
    return tf.nn.sigmoid(tf.add(tf.matmul(x,weight),bias))

def fully_connected_relu(x,weight,bias):
    return tf.nn.relu(tf.add(tf.matmul(x,weight),bias))

In [5]:
X = tf.placeholder(tf.float32,[None,n_dimensions])
Y = tf.placeholder(tf.float32,[None,n_classes])

weight_lone = weight_variable([n_dimensions,nhu_lone])
bias_lone = bias_variable([nhu_lone])

weight_ltwo = weight_variable([nhu_lone,nhu_ltwo])
bias_ltwo = bias_variable([nhu_ltwo])

weight_lout = weight_variable([nhu_ltwo,n_classes])
bias_lout = bias_variable([n_classes])

l1 = fully_connected_relu(X,weight_lone,bias_lone)
l2 = fully_connected_relu(l1,weight_ltwo,bias_ltwo)
y_ = fully_connected_sigmoid(l2,weight_lout,bias_lout,True)

In [6]:
loss_function = -tf.reduce_mean(((Y * tf.log(y_)) + ((1 - Y) * tf.log(1 - y_))))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

In [7]:
with tf.Session() as session:
    tf.initialize_all_variables().run()
    print"--------- Epoch -----------"
    for epoch in range(training_epochs):
        cost_history = np.empty((0),dtype=float)
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer,loss_function],feed_dict = {X: batch_x, Y: batch_y})
            cost_history = np.append(cost_history,c)
        print epoch, 
        
    y_pred = session.run(y_,feed_dict = { X: test_x })
    print "\n\nROC AUC Score: ",roc_auc_score(test_y,y_pred)
    
    y_pred[y_pred > 0.5] = 1 # considering probability above 0.5 as true label
    y_pred[y_pred < 0.5] = 0 # considering any probability below 0.5 as false label
    print "F1 Score: ", f1_score(test_y, y_pred, average='micro')  

--------- Epoch -----------
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

ROC AUC Score:  0.712481999143
F1 Score:  0.659800776633
